In [ ]:
import pickle
from pydantic import BaseModel, Field
from typing import Literal, Optional, List

In [5]:
from typing import Any, Dict, List, Optional, Literal, ForwardRef

class ToolArguments(BaseModel):
    keys: List[str] = Field(description="A list of arguments to a tool")
    values: List[str] = Field(description="A list of argument values to a tool")
    # arguments: Dict[str, Any] = Field(description="A dictionary where keys are tool arguments and values are the tool call values")


class ToolCall(BaseModel):
    """Represents a tool call request from the LLM."""

    id: str = Field(description="The ID of the tool call.")
    name: str = Field(description="The name of the tool to call.")
    arguments: ToolArguments = Field(description="The arguments to call the tool with.")


class ToolCalls(BaseModel):
    id: int = Field(description="An ID for the tool calls")
    tool_calls: List[ToolCall] = Field(
        description="A list of tools to be executed sequentially."
    )

class PlannerTask(BaseModel):
    """Represents a single task generated by the Planner."""

    id: int = Field(description="Sequential ID for the task.")
    description: str = Field(
        description="Clear description of the task to be executed."
    )
    tool_calls: List[ToolCall] = Field(
        description="A list of tools to be executed sequentially to complete the task"
    )
    status: Optional[
        Literal[
            "input_required",
            "completed",
            "error",
            "pending",
            "incomplete",
            "todo",
            "not_started",
        ]
    ] = Field(default="input_required", description="Status of the task")


class Plan(BaseModel):
    """Output schema for the Planner Agent."""

    original_query: str = Field(description="The original user query for context.")
    description: str = Field(description="Clear description of the overall plan.")
    tasks: List[PlannerTask] = Field(
        description="A list of tasks to be executed sequentially."
    )


class ToolResult(BaseModel):
    """Represents the result of a tool execution."""

    tool_call_id: str = Field(description="The ID of the tool call this result is for.")
    result: str = Field(description="The result of the tool execution.")
    is_error: bool = Field(
        default=False, description="Whether the tool execution resulted in an error."
    )


class ResponseFormat(BaseModel):
    """Respond to the user in this format."""

    status: Literal["input_required", "completed", "error"] = "input_required"
    question: str = Field(
        description="Input needed from the user to generate the code search plan"
    )
    content: Plan = Field(
        description="List of tasks when the code search plan is generated"
    )


class TaskExecutionResponse(BaseModel):
    """Represents a single task generated by the Planner."""

    id: int = Field(description="Id of task we are executing.")
    description: str = Field(description="Clear description of the task to be executed.")
    tools_sueggested: str = Field(description="A list of the tools suggested for the task")
    response_type: Optional[
        Literal[
            "tool_calls",
            "text",
        ]
    ] = Field(default="input_required", description="The response type of the task execution")
    tool_calls: List[ToolCall] = Field(description="A list of tool calls to be executed. Empty if response_type is text")


class ContentResponse(BaseModel):
    type: str = Field(
        description="The type of content such as body paragraph, introduction conclusion etc"
    )
    context_used: str = Field(description="The context used to generate the content")
    content: str = Field(description="The content generated by the agent")

In [7]:
with open("plan.pkl", "rb") as f:
    plan_data = pickle.load(f)

In [8]:
plan = Plan(**plan_data)

In [15]:
plan

Plan(original_query='write an essay on the culture impact of the internet', description='Plan to write an essay on the cultural impact of the internet with structured research, writing, and review tasks.', tasks=[PlannerTask(id=1, description='Research the cultural impact of the internet to gather relevant and accurate information for the essay.', tool_calls=[ToolCall(id='1', name='functions.wiki_search', arguments=ToolArguments(keys=['query', 'sentences'], values=['cultural impact of the internet', '5']))], status='todo'), PlannerTask(id=2, description='Write the introduction paragraph for the essay, outlining the main topic and its significance.', tool_calls=[ToolCall(id='2', name='functions.wiki_search', arguments=ToolArguments(keys=['query'], values=['introduction to cultural impact of the internet']))], status='todo'), PlannerTask(id=3, description='Write the first body paragraph discussing the positive cultural impacts of the internet.', tool_calls=[ToolCall(id='3', name='functio